In [1]:
%load_ext fortranmagic
%load_ext cython

C:\userdata\takuyai\Miniconda3\envs\cimdev\lib\site-packages\fortranmagic.py:147: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module since IPython 4.0.
  self._lib_dir = os.path.join(get_ipython_cache_dir(), 'fortran')


In [2]:
%%fortran

! dam discharge calculation test
subroutine ft_test(vol)
    real*8, intent(out) :: vol
    
    vol = 1.4920d-3 * (306505.0d0 - 306500.0d0)**1.5280d0
end subroutine

In [3]:
ft_test()

0.017449982259160068

In [4]:
def py_test():
    discharge = 0.001492 * (306505.0 - 306500.0)**1.5280
    return discharge

In [5]:
py_test()

0.017449982259160068

In [6]:
%timeit ft_test()

The slowest run took 23.53 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 63.8 ns per loop


In [7]:
%timeit py_test()

The slowest run took 14.26 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 84.3 ns per loop


In [8]:
%%cython
cpdef double cy_test():
    cdef double discharge = 0.001492 * (306505.0 - 306500.0)**1.5280
    return discharge

In [9]:
%timeit cy_test()

The slowest run took 13.78 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 87.2 ns per loop


In [10]:
# import numpy as np
# from libc.math import exp
from math import exp
def py_calc_interim_cmd(cmd, param_d, rainfall):
    """Calculate interim CMD (M_{f}) in its linear form.

    :param cmd: float, current Catchment Moisture Deficit (M_{k})
    :param param_d: float, model parameter factor `d`
    :param rainfall: float, rainfall for current time step in mm

    :returns: float, interim CMD (M_{f})
    """
    if cmd < param_d:
        Mf = cmd * exp(-rainfall / param_d)
    elif cmd < (param_d + rainfall):
        Mf = param_d * exp((-rainfall + cmd - param_d) / param_d)
    else:
        Mf = cmd - rainfall
    # End if

    return Mf
# End calc_interim_cmd()

In [11]:
%timeit py_calc_interim_cmd(200.0, 194.0, 16.5)

The slowest run took 12.69 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 308 ns per loop


In [12]:
%time py_calc_interim_cmd(200.0, 194.0, 16.5)

Wall time: 0 ns


183.779091697232

In [13]:
%%fortran

subroutine ft_calc_interim_cmd(cmd, param_d, rainfall, Mf)
    real*8 :: cmd, param_d, rainfall
    real*8, intent(out) :: Mf

    if (cmd.lt.param_d) then
        Mf = cmd * exp(-rainfall / param_d)
    else if (cmd.lt.(param_d + rainfall)) then
        Mf = param_d * exp((-rainfall + cmd - param_d) / param_d)
    else
        Mf = cmd - rainfall
    endif

end subroutine

In [14]:
%timeit ft_calc_interim_cmd(200.0, 194.0, 16.5)

The slowest run took 12.25 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 147 ns per loop


In [15]:
%time ft_calc_interim_cmd(200.0, 194.0, 16.5)

Wall time: 0 ns


183.779091697232

In [16]:
%%cython

from __future__ import division

from libc.math cimport exp

cpdef double cy_calc_interim_cmd(double cmd, double param_d, double rainfall):
    """Calculate interim CMD (M_{f}) in its linear form.

    :param cmd: float, current Catchment Moisture Deficit (M_{k})
    :param param_d: float, model parameter factor `d`
    :param rainfall: float, rainfall for current time step in mm

    :returns: float, interim CMD (M_{f})
    """
    cdef double Mf
    if cmd < param_d:
        Mf = cmd * exp(-rainfall / param_d)
    elif cmd < (param_d + rainfall):
        Mf = param_d * exp((-rainfall + cmd - param_d) / param_d)
    else:
        Mf = cmd - rainfall
    # End if

    return Mf
# End calc_interim_cmd()

In [17]:
%timeit cy_calc_interim_cmd(200.0, 194.0, 16.5)

The slowest run took 14.00 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 85.8 ns per loop


In [18]:
%time cy_calc_interim_cmd(200.0, 194.0, 16.5)

Wall time: 0 ns


183.779091697232

In [19]:
# %load_ext julia.magic

In [20]:
# %%julia
# 1+1


In [21]:
import math
import numpy as np

In [22]:
%timeit math.pi

The slowest run took 30.96 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 38.8 ns per loop


In [23]:
%timeit np.pi

10000000 loops, best of 3: 34.9 ns per loop


In [24]:
%timeit np.arctan(2.0 * 150.0)

The slowest run took 36.51 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 584 ns per loop


In [25]:
%timeit math.atan(2.0 * 150.0)

The slowest run took 24.58 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 97.8 ns per loop


In [26]:
%timeit math.tan(150)

The slowest run took 74.83 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 124 ns per loop


In [27]:
%timeit np.tan(150)

The slowest run took 23.06 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 743 ns per loop


In [28]:
def ret_tan(x):
    return math.tan(x)

In [29]:
%timeit ret_tan(150.0)

The slowest run took 16.90 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 178 ns per loop


In [30]:
%%fortran

subroutine ft_tan(x)
    real*8, intent(inout) :: x
            
    x = tan(x)

end subroutine

In [31]:
%timeit ft_tan(150.0)

The slowest run took 19.78 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 122 ns per loop


In [32]:
%%cython
from libc.math cimport tan

cpdef double cy_tan(double x):
    return tan(x)

In [33]:
%timeit cy_tan(150.0)

The slowest run took 22.00 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 68.3 ns per loop


In [34]:
%%fortran
subroutine ft_flow_calc(a, b, prev_quick, prev_slow, u, r, area, loss, pscale, quick, slow, flow, a2, b2, tmp_slow, tmp_flow)
    real*8, intent(in) :: a, b, prev_quick, prev_slow, u, r, area, loss, pscale
    real*8, intent(out) :: quick, slow, flow
    real*8, intent(out) :: a2, b2, tmp_flow, tmp_slow
    
    a2=0.5d0
    if ((prev_quick+u*area-0.5*loss).gt.0) then
        quick=1/(1+a)*(prev_quick+u*area-loss/2.0d0)
        flow=a*quick

    else
        if(loss.eq.0.0d0) then
            a2=0.0d0
        else
            a2=(prev_quick+u*area)/loss
            if(a2.gt.1) then
                a2=1.0d0
            endif
            if(a2.lt.0) then
                a2=0.0d0
            endif
        endif

        quick=prev_quick+u*area-a2*loss
        flow=0.0d0
    endif
    
    tmp_flow = flow

    b2=1.0d0-a2
    tmp_slow = (prev_slow+pscale*r*area-loss*b2)
    if ((prev_slow+r*area-b2*loss).gt.0) then
        slow=1/(1+b)*(prev_slow+pscale*r*area-loss*b2)
        flow=flow+b*slow
    else
        slow=slow+pscale*r*area-b2*loss
    endif

end subroutine

In [35]:
# a, b, prev_quick, prev_slow, u, r, area, loss, pscale
a = 54.0
b = 0.18
prev_quick = 24.0
prev_slow = 500.0
u = 0.1
r = 0.01
area = 1900
loss = 0.0
pscale = 0.0

In [36]:
def calc_ft_flows(prev_quick, prev_slow, e_rain, recharge, area, a, b, loss=0.0, pscale=0.0):
    """

    :param prev_quick: float, previous quickflow storage
    :param prev_slow: float, previous slowflow storage
    :param e_rain: float, effective rainfall in mm
    :param recharge: float, recharge amount in mm
    :param area: float, catchment area in km^2
    :param a: float, `a` factor controlling quickflow rate
    :param b: float, `b` factor controlling slowflow rate
    :param loss: float, losses in mm depth

    :returns: tuple[float], quick store, slow store, outflow
    """
    a2 = 0.5
    if (prev_quick + e_rain * area - 0.5 * loss) > 0.0:
        quick_store = 1.0 / (1.0 + a) * (prev_quick + e_rain * area - loss / 2.0)
        outflow = a * quick_store
    else:
        a2 = 0.0 if loss == 0.0 else max(0.0, min(1.0, prev_quick + e_rain * area / loss))
        quick_store = prev_quick + e_rain * area - a2 * loss
        outflow = 0.0
    # End if
    
    tmp_flow = outflow

    b2 = 1.0 - a2
    tmp_slow = (prev_slow+r*area-loss*b2)
    slow_store = prev_slow + recharge * area - loss * b2  # b2 * loss
    if (prev_slow + recharge * area - b2 * loss) > 0.0:
        slow_store = 1.0 / (1.0 + b) * slow_store
        outflow = outflow + b * slow_store
    # End if

    return quick_store, slow_store, outflow, a2, b2, tmp_slow, tmp_flow
# End calc_ft_flows()

In [37]:
ft_flow_calc(a, b, prev_quick, prev_slow, u, r, area, loss, pscale=1.0)  # , 0.0, 0.0

(3.8909090909090907,
 439.83050847457633,
 289.27858243451465,
 0.5,
 0.5,
 519.0,
 210.1090909090909)

In [38]:
calc_ft_flows(prev_quick, prev_slow, u, r, area, a, b, loss)

(3.8909090909090907,
 439.83050847457633,
 289.27858243451465,
 0.5,
 0.5,
 519.0,
 210.1090909090909)

In [39]:
slow_store = prev_slow + r * area - loss * 0.5
slow_store

519.0

In [40]:
from math import exp
def calc_ft_interim(cmd, rain, d, d2, alpha):
    """Direct port of original Fortran implementation to calculate interim CMD.

    :param cmd: float, Catchment Moisture Deficit
    :param rain: float, rainfall for time step in mm
    :param d: float, flow threshold value
    :param d2: float, scaling factor applied to `d`
    :param alpha: float, took value from IHACRESparams.csv file for 406219 for dev purposes only

    :returns: tuple[float], interim CMD value, effective rainfall, recharge (all in mm)
    """
    d2 = d * d2

    tmp_cmd = cmd
    e_rain = 0.0
    recharge = 0.0
    if rain == 0.0:
        return cmd, e_rain, recharge
    # End if

    if tmp_cmd > (d2 + rain):
        # CMD never reaches d2, so all rain is effective
        cmd = tmp_cmd - rain
    else:
        if tmp_cmd > d2:
            tmp_rain = rain - (tmp_cmd - d2)  # leftover rain after reaching d2 threshold
            tmp_cmd = d2
        else:
            tmp_rain = rain
        # End if

        d1a = d * (2.0 - exp(-(rain / 50.0)**2))
        if tmp_cmd > d1a:
            eps = d2 / (1.0 - alpha)

            # original comment: now get rainfall to reach cmd = d1a
            # amount of rain necessary to get to threshold `d`
            depth_to_d = eps * log((alpha + tmp_cmd / eps) / (alpha + d1a / eps))
            if depth_to_d > tmp_rain:
                lam = exp(tmp_rain * (1.0 - alpha) / d2)
                epsilon = alpha * eps

                cmd = tmp_cmd / lam - epsilon * (1.0 - 1.0 / lam)
                e_rain = 0.0
            else:
                if (tmp_cmd > d1a):
                    tmp_rain = tmp_rain - depth_to_d

                tmp_cmd = d1a
                gamma = (alpha * d2 + (1.0 - alpha) * d1a) / (d1a * d2)
                cmd = tmp_cmd * exp(-tmp_rain * gamma)
                e_rain = alpha * (tmp_rain + 1.0 / d1a / gamma * (cmd - tmp_cmd))
            # End if
        else:
            gamma = (alpha * d2 + (1.0 - alpha) * d1a) / (d1a * d2)
            cmd = tmp_cmd * exp(-tmp_rain * gamma)
            e_rain = alpha * (tmp_rain + 1.0 / d1a / gamma * (cmd - tmp_cmd))
        # End if

        recharge = rain - (tmp_cmd - cmd) - e_rain
    # End if

    return cmd, e_rain, recharge
# End calc_ft_interim()

In [41]:
%%fortran
subroutine ft_interim_cmd(d1, d2, CMD_old, rain, alpha, cmd, u, r)
    ! local
    real*8 d1a, cmd2, epsilonn, rain3, gamma
    
    ! output
    real*8, intent(out) :: cmd, u, r
    
    
    d1a=d1*(2.0d0-exp(-(rain/50.0d0)**2))
    cmd2 = CMD_old
    if (rain.eq.0) then
        u=0.0d0
        r=0.0d0
        cmd=cmd2
    else
        if (cmd2.gt.d2+rain) then
            cmd = cmd2-rain
            u = 0.0
            r = 0.0
        else
            if (cmd2.gt.d2) then
                rain2 = rain-(cmd2-d2)
                cmd2 = d2
            else
                rain2 = rain
            endif
            
            if (cmd2.gt.d1a) then
                epsilonn = d2/(1.0-alpha)
                rain3=epsilonn*log((alpha+cmd2/epsilonn)/(alpha+d1a/epsilonn))
                if (rain3.ge.rain2) then
                    lamda = exp(rain2*(1.0-alpha)/d2)
                    epsilon = alpha*epsilonn
                    cmd = cmd2/lamda-epsilon*(1.0-1.0/lamda)
                    u = 0.0
                else
                    if (cmd2.gt.d1a) then
                        rain2 = rain2-rain3
                    endif
                    cmd2 = d1a
                    gamma = (alpha*d2+(1-alpha)*d1a)/(d1a*d2)
                    cmd = cmd2*exp(-rain2*gamma)
                    u = alpha*(rain2+1.0/d1a/gamma*(cmd-cmd2))
                endif
            else
                gamma = (alpha*d2+(1-alpha)*d1a)/(d1a*d2)
                cmd = cmd2*exp(-rain2*gamma)
                u = alpha*(rain2+1.0/d1a/gamma*(cmd-cmd2))
            endif
            r = rain-(CMD_old-cmd)-u
        endif
    endif
    
    ! if (cmd.gt.f) then
    !     et = e*evap*exp( (1-cmd/f)*2)
    ! else
    !     et = e*evap
    ! endif
end subroutine

Running...
   C:\userdata\takuyai\Miniconda3\envs\cimdev\python.exe -m numpy.f2py -m _fortran_magic_068b2a456e820b17930d25a7b301aaa8 -c C:\Users\takuyai\.ipython\fortran\_fortran_magic_068b2a456e820b17930d25a7b301aaa8.f90
running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "_fortran_magic_068b2a456e820b17930d25a7b301aaa8" sources
f2py options: []
f2py:> c:\users\takuyai\appdata\local\temp\tmpziwkgg\src.win-amd64-2.7\_fortran_magic_068b2a456e820b17930d25a7b301aaa8module.c
creating c:\users\takuyai\appdata\local\temp\tmpziwkgg\src.win-amd64-2.7
Reading fortran codes...
	Reading file 'C:\\Users\\takuyai\\.ipython\\fortran\\_fortran_magic_068b2a456e820b17930d25a7b301aaa8.f90' (format:free)
Post-processing...
	Block: _fortran_magic_068b2a456e820b17930d25a7b301aaa8
{}
In: :_fortran


Ok. The following fortran objects are ready to use: ft_interim_cmd


In [65]:
cmd = 100.0
d = 150.0
d2 = 2.0
e = 1.0
f = 0.8
rain = 6.0
evap = 3.0

In [66]:
ft_interim_cmd(d1=d, d2=d*d2, cmd_old=cmd, alpha=0.727, rain=rain)

(96.6445752611055, 1.5433629240991225, 1.1012123370063789)

In [67]:
# Python implementation
# cmd, rain, d, d2, alpha
py_mf, py_u, py_r = calc_ft_interim(cmd=cmd, rain=rain, d=d, d2=d2, alpha=0.727)

In [68]:
from pyIHACRES import ihacres_funcs as ifuncs

In [69]:
ifuncs.calc_ET(e, evap, py_mf, f, d)

3.0

In [70]:
if cmd > (d * d2):
    et = e * evap * exp( (1 - py_mf / (d * d2))*2.0)
else:
    et = e * evap
    
et

3.0

In [71]:
ifuncs.calc_cmd(cmd, rain, et, py_u, py_r)

99.64457534233316

In [73]:
cmd = 100.0 + et + py_u + py_r - rain
cmd

99.64457534233316